# Cluster Analysis

Now that I have made a final decision on which clustering algorithm to use, I can:

1. scale it up to include all of the data.

2. Analyze differences in the clusters for insights

## Scaling up the clustering algorithm

Previously I had to seriously scale down the data being fed the clustering algorithms to avoid memory error and make reasonable comparisons. Now I will run the Gaussian Clustering again with the maximum data.

#### Data Prep

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

In [3]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [4]:
#Import to dataframe
df_features = pd.read_csv('processed_hotel_bookings.csv')

##### Dimensionality reduction

Elaboration of my dimensionality reduction configuration decision process is given in the Final Dimension Reduction Configuration notebook.

In [5]:
#Libraries
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import normalize

In [6]:
# Loading the data
X = df_features
y = df_features.is_canceled

# Standarizing the features
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

X_norm = normalize(X_std)

In [7]:
from sklearn.manifold import TSNE
import time

In [8]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=100, n_iter=300)
tsne_results = tsne.fit_transform(X_std)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

[t-SNE] Computing 301 nearest neighbors...
[t-SNE] Indexed 39568 samples in 0.604s...
[t-SNE] Computed neighbors for 39568 samples in 16.916s...
[t-SNE] Computed conditional probabilities for sample 1000 / 39568
[t-SNE] Computed conditional probabilities for sample 2000 / 39568
[t-SNE] Computed conditional probabilities for sample 3000 / 39568
[t-SNE] Computed conditional probabilities for sample 4000 / 39568
[t-SNE] Computed conditional probabilities for sample 5000 / 39568
[t-SNE] Computed conditional probabilities for sample 6000 / 39568
[t-SNE] Computed conditional probabilities for sample 7000 / 39568
[t-SNE] Computed conditional probabilities for sample 8000 / 39568
[t-SNE] Computed conditional probabilities for sample 9000 / 39568
[t-SNE] Computed conditional probabilities for sample 10000 / 39568
[t-SNE] Computed conditional probabilities for sample 11000 / 39568
[t-SNE] Computed conditional probabilities for sample 12000 / 39568
[t-SNE] Computed conditional probabilities for s

## Gaussian

Fortunately I was able to include all of the data this time. Only 172 observations out of nearly 40,000 were not cleanly
clustered.

In [9]:
#Libraries
from sklearn.mixture import GaussianMixture
from matplotlib.patches import Ellipse

In [10]:
# Defining the agglomerative clustering
gmm_cluster = GaussianMixture(n_components=2, random_state=123, covariance_type='diag')
_
# Fit model
G_clusters = gmm_cluster.fit_predict(X_std)

### As a reminder, below blue is for the bookings that were canceled, and red is for the bookings that were not canceled.m

In [11]:
plt.figure(figsize=(10,5))
colours = 'rbg'
for i in range(tsne_results.shape[0]):
    plt.text(tsne_results[i, 0], tsne_results[i, 1], str(G_clusters[i]),
             color=colours[y[i]],
             fontdict={'weight': 'bold', 'size': 50}
        )

plt.xticks([])
plt.yticks([])
plt.axis('off')
plt.show()

## The "zero" cluster maps cleanly to the bookings that were not canceled. The "one" cluster maps nearly cleanly to the bookings that were canceled.

In [12]:
G_clusters = np.array([0 if x == 1 else 1 if x==0 else 2 for x in G_clusters])

pd.crosstab(y, G_clusters)

col_0,0,1
is_canceled,,
0,28319,172
1,0,11077


### Here we can see that the same features that were selected for the random forest model show noteworthy and sometimes stark differences between clusters

Median values for agent_cancel_score, country_cancel_score, and lead_time sharply differed between clusters.

In [14]:
df_features['G_clusters'] = G_clusters

df_features.groupby('G_clusters').mean()

,Unnamed: 0,is_canceled,deposit_score,agent_cancel_score,country_cancel_score,lead_time,tf_lead_time,market_segment_score,required_car_parking_spaces,agent_market_int
G_clusters,,,,,,,,,,
0,24108.033476,0.00000,0.249188,0.246350,0.254555,79.287793,2.372310,0.261061,0.189096,0.072627
1,8934.760068,0.98471,0.356783,0.363851,0.343264,129.055116,3.103728,0.326738,0.000089,0.125593


In [15]:
df_features.groupby('G_clusters').median()

,Unnamed: 0,is_canceled,deposit_score,agent_cancel_score,country_cancel_score,lead_time,tf_lead_time,market_segment_score,required_car_parking_spaces,agent_market_int
G_clusters,,,,,,,,,,
0,25378,0,0.249238,0.179748,0.215062,39,2.498999,0.353894,0,0.048614
1,9192,1,0.249238,0.395770,0.421894,110,3.238532,0.353894,0,0.140061


In [17]:
df_features.groupby('G_clusters').min()

,Unnamed: 0,is_canceled,deposit_score,agent_cancel_score,country_cancel_score,lead_time,tf_lead_time,market_segment_score,required_car_parking_spaces,agent_market_int
G_clusters,,,,,,,,,,
0,0,0,0.15493,0.0,0.000000,0,0.0,0.13607,0,0.0
1,8,0,0.15493,0.0,0.021739,0,0.0,0.13607,0,0.0


In [18]:
df_features.groupby('G_clusters').max()

,Unnamed: 0,is_canceled,deposit_score,agent_cancel_score,country_cancel_score,lead_time,tf_lead_time,market_segment_score,required_car_parking_spaces,agent_market_int
G_clusters,,,,,,,,,,
0,39595,0,0.249238,0.925926,0.75,737,5.210350,0.424211,1,0.331729
1,38309,1,0.959860,1.000000,1.00,471,4.658598,0.424211,1,0.424211


### Although this model is very well performing, it's strength is actually it's weakness. All we have done is created a single cluster for cancelled bookings and a single cluster for bookings that were not canceled. 

### We will need to examine a qualitatively if not quantitatively better model to get the insights we need.